In [1]:
from importlib import reload
from astropy import units as u
from astropy import constants as const
from astropy.modeling.models import BlackBody
from astropy.units.equivalencies import spectral

import numpy as np
import matplotlib.pyplot as plt

from scipy.special import expn
from scipy.integrate import simpson

from src import astro530
from src import N_integrator

In [3]:
simpson_wrapper = N_integrator.simpson_wrapper

## Problem 6) Problem 3: Redman’s eclipse spectrum
### from Rutten without a, g, h

This problem illustrates the transition from optically thick to optically thin line formation
near the solar limb. R.O. Redman obtained a spectrum of the Ca II K line at λ$_K$ = 393.3 nm
during a total solar eclipse. A tracing made from his plate is sketched in Fig. 10.10. The K
line shows four peaks which we try to explain.

The density in the solar atmosphere drops with height h as N ∼ exp(−h/H), with scale
height H ≈ 100 km. Assume that the ratio of line-center extinction over continuum extinction η = α$^l$
/α$^c$ has η$_K$ = 106 at all heights. Assume also, for the moment, that the
atmosphere is isothermal with T = 6000 K for all h and that LTE holds.

b) Now the geometry (Fig. 10.11). Let us use τ for radial optical depth and t for optical
thickness along the beam. The zero point of the radial height scale h = 0 km is defined
as the location with radial optical depth τ c
1 = 1 in the continuum close to the K line,
i.e., measured along line of sight number 1. Why is this a logical definition? Where is
τ c
1 = 0? What is the optical thickness of the sun along this line of sight?

The lines of sight 2 and 3 are tangential to the sun. The solar limb is defined to be the
location hi at which the continuum optical thickness t
c
i of the sun along a tangential line
of sight i equals unity: that direction i for which t
c
i = 1. In the continuum near the K
line this is number 2, cutting through shells with h > 300 km and touching the limb at
hL = h2 = 300 km. Why isn’t the limb at h = 0 km?

Are there wavelengths in the spectrum with hL < 300 km?
Line of sight 3 is tangential to the shell with h3 = 600 km. Estimate the continuum
optical thickness tc
3 of the sun for this viewing direction.

## Problem 7) Hν (0)
Use your work from Problem 5 to write a function that calculates Hν (0) given an array of
optical depths τν using an exponential integral. In addition to an array of optical depths,
the function should take as an argument the name of another function that returns the
relevant source function at those optical depths (or whatever the equivalent is in the
language you are using). Choose a quadratic form for Sν (τν ) as in problem 1a) and
compute Hν (0) for it.

Confirm that this works as you expect it to for a linear source function.

You are now just a source function and an optical depth grid away from calculating the
emergent flux density of stars at a given wavelength.


In [3]:
def source_func(t, a0 = 1, a1 = 1, a2 = 1):
    """ Radiative transfer source function
    Given τ_ν as a single value or an array, and key word values for a_n where 
    n < 3, the function will output the source function value. This is viable 
    up to quadratic form.
    
    input parameters:
    
    t [float or array]: values for τ_ν, the optical depth.
    a0 [float]: Named variable for the zeroth order source term
    a1 [float]: Named variable for the linear source term
    a2 [float]: Named variable for the quadratic source term. Set to zero for a 
                linear source function.
                
    output values:
    
    S_ν [float or array]: outputs the value(s) for the source function. 
    
    """
    return a0+a1*t+a2*t**2

def SvxEn(tval, tau = 0, n=2, src_func = source_func, **kwargs):
    S = src_func(tval, **kwargs)
    
    if n == 1 and abs(tval-tau) == 0: return S*229.7  # E1(1e-100)
    
    En = expn(n,abs(tval-tau))
    
    return S*En

def astrophysical_flux(t_arr, tmin = 0, tmax = 1e2, n_size = 1e-5, src_func = source_func, 
                       int_wrapper=simpson_wrapper, **kwargs):
    """ Astrophysical Flux H_ν(t)
    Given an array of τ_ν, an optional function variable and keywords for said 
    function, this function will output the astrophysical flux at zero optical 
    depth as calculated using numerical integration with scipy.integrate.simpson  
    
    input parameters:
    
    t_arr [array-like]: values for optical depth from 0 to infinity.
    src_dunc [function]: name of the source function's function which is given 
                         at least optical depth values in array form. 
    int_wrapper [function]: name of the integration wrapper function which will 
                            numerically solve the flux problem. Make sure the 
                            function's inputs follow the same format as 
                            N_integrator.simpson_wrapper. 
    kwargs: keyword arguments for the source function and integrator.
    
    output values:
    
    H_ν(0) [float]: Outputs the value for the astrophysical flux at a τ_ν = 0.
    """
    t_arr = np.array(t_arr)
#     S_arr = src_func(t_arr,**kwargs)
#     E2_arr = expn(2,t_arr)
    
#     y_arr = np.array(S_arr * E2_arr)
    
    Hv = np.zeros(len(t_arr))
    
    for i in range(len(t_arr)):
        tv = t_arr[i]
        
        if tv < 0: raise ValueError("Optical depth cannot be negative. Fix index = "+str(i))
        
        outward = int_wrapper(tv,tmax,n_size = n_size, scale = "log", function=SvxEn, 
                                  tau = tv, n = 2, src_func = src_func, **kwargs)
        inward = -int_wrapper(tmin,tv,n_size = n_size, scale = "log", function=SvxEn, 
                                      tau = tv, n = 2, src_func=src_func, **kwargs)
        Hv[i] = 1/2*(outward+inward)
    #return y_arr
    
    return Hv

In [11]:
3/2*(astrophysical_flux([0])[0]-2/3)

1.6653345369377348e-16

In [13]:
(astrophysical_flux([0],a2=0)[0]-5/12)*12/5

-1.3322676295501878e-16